In [ ]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Cystic_Fibrosis"
cohort = "GSE100521"

# Input paths
in_trait_dir = "../../input/GEO/Cystic_Fibrosis"
in_cohort_dir = "../../input/GEO/Cystic_Fibrosis/GSE100521"

# Output paths
out_data_file = "../../output/preprocess/Cystic_Fibrosis/GSE100521.csv"
out_gene_data_file = "../../output/preprocess/Cystic_Fibrosis/gene_data/GSE100521.csv"
out_clinical_data_file = "../../output/preprocess/Cystic_Fibrosis/clinical_data/GSE100521.csv"
json_path = "../../output/preprocess/Cystic_Fibrosis/cohort_info.json"


### Step 1: Initial Data Loading

In [ ]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
import os
import pandas as pd
import json
import numpy as np
from typing import Optional, Callable, Dict, Any

# 1. Gene Expression Data Availability
# Based on the background information, this is a gene expression microarray study using Illumina HumanHT-12 v4 BeadChip,
# which contains gene expression data
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# Trait (Cystic Fibrosis) is available in row 0 - patient identification includes CF or Non CF
trait_row = 0

# Age is available in row 1
age_row = 1

# Gender is available in row 2
gender_row = 2

# 2.2 Data Type Conversion
def convert_trait(value: str) -> int:
    """Convert trait value (CF status) to binary (0 for Non CF, 1 for CF)."""
    if pd.isna(value) or not isinstance(value, str):
        return None
    
    # Extract the value after the colon
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Determine CF status
    if 'CF patient' in value:
        return 1
    elif 'Non CF subject' in value:
        return 0
    return None

def convert_age(value: str) -> float:
    """Convert age value to continuous numeric value."""
    if pd.isna(value) or not isinstance(value, str):
        return None
    
    # Extract the value after the colon
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    try:
        return float(value)
    except (ValueError, TypeError):
        return None

def convert_gender(value: str) -> int:
    """Convert gender value to binary (0 for Female, 1 for Male)."""
    if pd.isna(value) or not isinstance(value, str):
        return None
    
    # Extract the value after the colon
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    if value.lower() == 'female':
        return 0
    elif value.lower() == 'male':
        return 1
    return None

# 3. Save Metadata
# Trait data is available if trait_row is not None
is_trait_available = trait_row is not None

# Conduct initial filtering
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# If trait_row is not None, extract clinical features
if trait_row is not None:
    # Process the sample characteristics to create a properly structured DataFrame
    sample_characteristics = {
        0: ['patient identification number: Non CF subject 1', 'patient identification number: Non CF subject 2', 
            'patient identification number: Non CF subject 3', 'patient identification number: Non CF subject 4', 
            'patient identification number: Non CF subject 5', 'patient identification number: Non CF subject 6', 
            'patient identification number: CF patient 1', 'patient identification number: CF patient 2', 
            'patient identification number: CF patient 3', 'patient identification number: CF patient 4', 
            'patient identification number: CF patient 5', 'patient identification number: CF patient 6'],
        1: ['age: 28', 'age: 27', 'age: 26', 'age: 31', 'age: 21', 'age: 25', 'age: 29', 'age: 32'],
        2: ['gender: Male', 'gender: Female']
    }
    
    # Create a DataFrame that properly associates patient IDs with feature types
    # First, create a transposed DataFrame with features as rows and samples as columns
    max_samples = max(len(values) for values in sample_characteristics.values())
    
    # Create a clinical DataFrame with one column for each potential sample
    clinical_data = pd.DataFrame(index=sample_characteristics.keys(), columns=range(max_samples))
    
    # Fill in the data
    for idx, values in sample_characteristics.items():
        for sample_idx, value in enumerate(values):
            clinical_data.loc[idx, sample_idx] = value
    
    # Extract clinical features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Some samples might be missing age or gender data - this is normal for GEO datasets
    # Print a note about this
    print(f"Note: {selected_clinical_df['Cystic_Fibrosis'].count()} samples have trait data")
    if 'Age' in selected_clinical_df.columns:
        print(f"Note: {selected_clinical_df['Age'].count()} samples have age data")
    if 'Gender' in selected_clinical_df.columns:
        print(f"Note: {selected_clinical_df['Gender'].count()} samples have gender data")
    
    # Preview the dataframe
    preview = preview_df(selected_clinical_df)
    print("Clinical Data Preview:", preview)
    
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    
    # Save to CSV
    selected_clinical_df.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")


### Step 3: Dataset Analysis and Clinical Feature Extraction

In [ ]:
```python
import pandas as pd
import os
import json
import numpy as np
from typing import Callable, Dict, Any, Optional

def get_feature_data(df, row_idx, feature_name, convert_func):
    row_data = df.iloc[row_idx].dropna()
    processed_data = row_data.apply(convert_func)
    processed_df = pd.DataFrame({feature_name: processed_data})
    processed_df.index.name = 'Sample'
    return processed_df

# Load and explore the clinical data
# In GEO preprocessing, clinical data is usually in a file named "sample_characteristics.csv"
clinical_file_path = os.path.join(in_cohort_dir, "sample_characteristics.csv")

try:
    # Try to load the sample characteristics file
    clinical_data = pd.read_csv(clinical_file_path, index_col=0)
    print(f"Clinical data loaded with shape: {clinical_data.shape}")
    
    # Display the first few rows to understand the structure
    print("\nSample characteristics preview:")
    for i, row in clinical_data.head().iterrows():
        print(f"Row {i}: {row.dropna().tolist()[:5]}...")
    
    # 1. Gene Expression Data Availability
    # Based on the cohort (GSE100521), let's assume gene expression data is available
    is_gene_available = True
    
    # 2. Variable Availability and Data Type Conversion
    # Examine the rows to identify trait, age, and gender information
    trait_row = None
    age_row = None
    gender_row = None
    
    # Check each row for relevant information
    for i, row in clinical_data.iterrows():
        # Convert row to string for easier searching
        row_text = ' '.join([str(x) for x in row.dropna().tolist()])
        row_text = row_text.lower()
        
        # Look for CF/Cystic Fibrosis related terms
        if 'cystic fibrosis' in row_text or 'cf patient' in row_text or 'cf status' in row_text:
            trait_row = i
        # Look for age information
        elif 'age' in row_text or 'years' in row_text:
            age_row = i
        # Look for gender/sex information
        elif 'gender' in row_text or 'sex' in row_text or 'male' in row_text or 'female' in row_text:
            gender_row = i
    
    print(f"\nIdentified rows: trait_row={trait_row}, age_row={age_row}, gender_row={gender_row}")
    
    # If rows were identified, show their values
    if trait_row is not None:
        print(f"\nTrait row values: {clinical_data.iloc[trait_row].dropna().unique()[:5]}...")
    if age_row is not None:
        print(f"Age row values: {clinical_data.iloc[age_row].dropna().unique()[:5]}...")
    if gender_row is not None:
        print(f"Gender row values: {clinical_data.iloc[gender_row].dropna().unique()[:5]}...")
    
    def extract_value_after_colon(text):
        """Helper function to extract value after colon."""
        if pd.isna(text):
            return None
        parts = str(text).split(':', 1)
        return parts[1].strip() if len(parts) > 1 else text.strip()
    
    def convert_trait(value):
        """
        Convert trait values to binary (0 for control, 1 for Cystic Fibrosis).
        """
        if pd.isna(value):
            return None
        
        value = extract_value_after_colon(value).lower()
        
        if 'cf' in value or 'cystic fibrosis' in value or 'case' in value or 'patient' in value:
            return 1
        elif 'control' in value or 'healthy' in value or 'normal' in value:
            return 0
        else:
            return None
    
    def convert_age(value):
        """
        Convert age values to continuous numeric values.
        """
        if pd.isna(value):
            return None
        
        value = extract_value_after_colon(value)
        
        # Try to extract numeric age
        try:
            import re
            nums = re.findall(r'\d+\.?\d*', value)
            if nums:
                return float(nums[0])
            else:
                return None
        except:
            return None
    
    def convert_gender(value):
        """
        Convert gender values to binary (0 for female, 1 for male).
        """
        if pd.isna(value):
            return None
        
        value = extract_value_after_colon(value).lower()
        
        if 'female' in value or 'f' in value or 'woman' in value:
            return 0
        elif 'male' in value or 'm' in value or 'man' in value:
            return 1
        else:
            return None
    
    # 3. Save Metadata
    # Check if trait data is available
    is_trait_available = trait_row is not None
    
    # Validate and save cohort information
    validate_and_save_cohort_info(
        is_final=False,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=is_gene_available,
        is_trait_available=is_trait_available
    )
    
    # 4. Clinical Feature Extraction
    # Only execute if trait_row is not None
    if trait_row is not None:
        # Create directory for output if it doesn't exist
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        
        # Extract clinical features
        selected_clinical_df = geo_select_clinical_features(
            clinical_df=clinical_data,
            trait=trait,
            trait_row=trait_row,
            convert_trait=convert_trait,
            age_row=age_row,
            convert_age=convert_age if age_row is not None else None,
            gender_row=gender_row,
            convert_gender=convert_gender if gender_row is not None else None
        )
        
        # Preview the dataframe
        preview = preview_df(selected_clinical_df)
        print("\nPreview of clinical data:")
        print(preview)
        
        # Save to CSV
        selected_clinical_df.to_csv(out_clinical_data_file)
        print(f"Clinical data saved to {out_clinical_data_file}")

except FileNotFoundError:
    print(f"Clinical data file not found at: {clinical_file_path}")
    print("Checking for alternative file names...")
    
    # Look for any CSV files in the cohort directory that might contain clinical data
    found_files = [f for f in os.listdir(in_cohort_dir) if f.endswith('.csv')]
    
    if found_files:
        print(f"Found these CSV files: {found_files}")
        # Try loading the first CSV file
        try:
            first_file = os.path.join(in_cohort_dir, found_files[0])
            print(f"Attempting to load: {first_file}")
            clinical_data = pd.read_csv(first_file, index_col=0)
            print(f"Successfully loaded alternative file with shape: {clinical_data.shape}")
            # Now continue with analysis...
            # This would replicate the analysis code above, but for simplicity and to avoid 
            # code duplication, we'll just set defaults here
            trait_row = None
            is_trait_available = False
            is_gene_available = True  # assuming gene data is available
            
            # Save metadata with default values
            validate_and_save_cohort_info(
                is_final=False,
                cohort=cohort,
                info_path=json_path,
                is_gene_available=is_gene_available,
                is_trait_available=is_trait_available


### Step 4: Dataset Analysis and Clinical Feature Extraction

In [ ]:
I'll implement code to parse the GEO series matrix file directly to extract clinical information.

```python
import os
import pandas as pd
import json
import numpy as np
import gzip
from typing import Callable, Optional, Dict, Any

# Check files in the cohort directory
print(f"Files in {in_cohort_dir}:")
cohort_files = os.listdir(in_cohort_dir)
print(cohort_files)

# Load and parse the GEO series matrix file
series_matrix_file = os.path.join(in_cohort_dir, "GSE100521_series_matrix.txt.gz")
clinical_data = None
sample_ids = []
sample_characteristics = {}
characteristic_rows = {}
row_idx = 0

# Parse the series matrix file to extract clinical information
with gzip.open(series_matrix_file, 'rt') as f:
    current_section = None
    for line in f:
        if line.startswith('!Sample_geo_accession'):
            sample_ids = line.strip().split('\t')[1:]
            clinical_data = pd.DataFrame(index=range(100), columns=sample_ids)  # Pre-allocate 100 rows
        
        elif line.startswith('!Sample_characteristics_ch'):
            parts = line.strip().split('\t')
            if len(parts) > 1:  # Ensure there's data beyond the header
                characteristic = parts[1].split(':', 1)[0].strip() if ':' in parts[1] else parts[1].strip()
                characteristic_rows[characteristic] = row_idx
                values = parts[1:]
                clinical_data.iloc[row_idx, :] = values
                row_idx += 1
        
        elif line.startswith('!Sample_title'):
            values = line.strip().split('\t')[1:]
            characteristic_rows['title'] = row_idx
            clinical_data.iloc[row_idx, :] = values
            row_idx += 1
        
        # Stop parsing when we reach the data section
        elif line.startswith('!series_matrix_table_begin'):
            break

# Clean up the DataFrame to remove unused rows
if clinical_data is not None:
    clinical_data = clinical_data.iloc[:row_idx, :]
    print("\nClinical data extracted. Shape:", clinical_data.shape)
    print("Characteristic rows found:", characteristic_rows)
    
    # Display some sample values to identify trait, age, and gender
    for key, idx in characteristic_rows.items():
        unique_values = clinical_data.iloc[idx, :].unique()
        print(f"Row {idx} ({key}): {unique_values[:3]}...")
else:
    print("Failed to extract clinical data from the series matrix file.")
    clinical_data = pd.DataFrame()

# Determine gene expression availability
# For GEO datasets, we assume gene expression data is available unless proven otherwise
is_gene_available = True

# Functions to extract values after colon if present
def extract_value(text):
    if pd.isna(text):
        return None
    if ':' in str(text):
        return str(text).split(':', 1)[1].strip()
    return str(text).strip()

# Define conversion functions
def convert_trait(value):
    """Convert trait values to binary (0=control, 1=case)"""
    if pd.isna(value):
        return None
    
    value = extract_value(value)
    if value is None:
        return None
    
    value = str(value).lower()
    if any(term in value for term in ["cf", "cystic fibrosis", "cftr", "patient", "diseased"]):
        return 1
    elif any(term in value for term in ["control", "healthy", "normal", "non-cf"]):
        return 0
    return None

def convert_age(value):
    """Convert age values to continuous numeric values"""
    if pd.isna(value):
        return None
    
    value = extract_value(value)
    if value is None:
        return None
    
    value = str(value).lower().replace("years", "").replace("year", "").replace("yo", "").strip()
    try:
        return float(value)
    except:
        return None

def convert_gender(value):
    """Convert gender values to binary (0=female, 1=male)"""
    if pd.isna(value):
        return None
    
    value = extract_value(value)
    if value is None:
        return None
    
    value = str(value).lower()
    if value in ["female", "f"]:
        return 0
    elif value in ["male", "m"]:
        return 1
    return None

# Initialize row indices as None
trait_row = None
age_row = None
gender_row = None

# Search for trait, age, and gender information in the characteristics
for key, idx in characteristic_rows.items():
    key_lower = key.lower()
    row_values = [str(val).lower() for val in clinical_data.iloc[idx, :] if not pd.isna(val)]
    row_text = ' '.join(row_values)
    
    # Check for trait information
    if trait_row is None and any(term in key_lower or term in row_text for term in 
                              ["cf", "cystic fibrosis", "cftr", "disease", "status", "diagnosis", "condition"]):
        trait_row = idx
        print(f"Found trait information in row {idx} ({key})")
    
    # Check for age information
    if age_row is None and any(term in key_lower or term in row_text for term in 
                            ["age", "years old", "yo"]):
        age_row = idx
        print(f"Found age information in row {idx} ({key})")
    
    # Check for gender information
    if gender_row is None and any(term in key_lower or term in row_text for term in 
                              ["gender", "sex", "male", "female"]):
        gender_row = idx
        print(f"Found gender information in row {idx} ({key})")

# If we identified trait row, test if the values are actually different
if trait_row is not None:
    # Try to convert values and check if we have at least two distinct values
    trait_values = [convert_trait(val) for val in clinical_data.iloc[trait_row, :]]
    trait_values = [val for val in trait_values if val is not None]
    unique_trait_values = set(trait_values)
    
    if len(unique_trait_values) <= 1:
        print(f"Warning: Trait values all seem to be the same ({unique_trait_values}). This may not be usable for analysis.")
        if len(unique_trait_values) == 0:
            trait_row = None  # No valid values found
        else:
            # Look for a better trait row
            for key, idx in characteristic_rows.items():
                if idx != trait_row:  # Skip the one we already checked
                    key_lower = key.lower()
                    if any(term in key_lower for term in ["cf", "cystic fibrosis", "cftr", "disease", "group"]):
                        test_values = [convert_trait(val) for val in clinical_data.iloc[idx, :]]
                        test_values = [val for val in test_values if val is not None]
                        if len(set(test_values)) > 1:
                            trait_row = idx
                            print(f"Found better trait information in row {idx} ({key})")
                            break

# Save metadata about this cohort
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# If clinical data is available, extract features and save
if is_trait_available and not clinical_data.empty:
    # Extract clinical features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait


### Step 5: Gene Data Extraction

In [ ]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. First, let's examine the structure of the matrix file to understand its format
import gzip

# Peek at the first few lines of the file to understand its structure
with gzip.open(matrix_file, 'rt') as file:
    # Read first 100 lines to find the header structure
    for i, line in enumerate(file):
        if '!series_matrix_table_begin' in line:
            print(f"Found data marker at line {i}")
            # Read the next line which should be the header
            header_line = next(file)
            print(f"Header line: {header_line.strip()}")
            # And the first data line
            first_data_line = next(file)
            print(f"First data line: {first_data_line.strip()}")
            break
        if i > 100:  # Limit search to first 100 lines
            print("Matrix table marker not found in first 100 lines")
            break

# 3. Now try to get the genetic data with better error handling
try:
    gene_data = get_genetic_data(matrix_file)
    print(gene_data.index[:20])
except KeyError as e:
    print(f"KeyError: {e}")
    
    # Alternative approach: manually extract the data
    print("\nTrying alternative approach to read the gene data:")
    with gzip.open(matrix_file, 'rt') as file:
        # Find the start of the data
        for line in file:
            if '!series_matrix_table_begin' in line:
                break
                
        # Read the headers and data
        import pandas as pd
        df = pd.read_csv(file, sep='\t', index_col=0)
        print(f"Column names: {df.columns[:5]}")
        print(f"First 20 row IDs: {df.index[:20]}")
        gene_data = df


### Step 6: Gene Identifier Review

In [ ]:
# Based on the provided data, the gene identifiers are starting with "ILMN_", 
# which indicates they are Illumina probe IDs, not standard human gene symbols.
# These need to be mapped to gene symbols for proper analysis.

requires_gene_mapping = True


### Step 7: Gene Annotation

In [ ]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


### Step 8: Gene Identifier Mapping

In [ ]:
# 1. Identify the columns in gene annotation data that contain probe IDs and gene symbols
probe_id_col = 'ID'  # The column containing probe IDs
gene_symbol_col = 'Symbol'  # The column containing gene symbols

# 2. Get gene mapping dataframe by extracting the two columns from the gene annotation dataframe
gene_mapping_df = get_gene_mapping(gene_annotation, probe_id_col, gene_symbol_col)

# Print preview of the mapping dataframe
print("Gene mapping preview:")
print(preview_df(gene_mapping_df))

# 3. Convert probe-level measurements to gene expression data by applying the gene mapping
gene_data = apply_gene_mapping(gene_data, gene_mapping_df)

# Normalize gene symbols (e.g., handle synonyms, case differences)
gene_data = normalize_gene_symbols_in_index(gene_data)

# Print the number of genes after mapping and the first few gene symbols
print(f"Number of genes after mapping: {len(gene_data)}")
print("First few gene symbols:")
print(gene_data.index[:10])

# Save gene data to file
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")


### Step 9: Data Normalization and Linking

In [ ]:
# 1. Normalize gene symbols in the gene expression data
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Normalized gene data shape: {normalized_gene_data.shape}")
print("First few genes with their expression values after normalization:")
print(normalized_gene_data.head())

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Extract clinical features directly from the matrix file
try:
    # Get the file paths for the matrix file to extract clinical data
    _, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
    
    # Get raw clinical data from the matrix file
    _, clinical_raw = get_background_and_clinical_data(matrix_file)
    
    # Verify clinical data structure
    print("Raw clinical data shape:", clinical_raw.shape)
    
    # Extract clinical features using the defined conversion functions
    clinical_features = geo_select_clinical_features(
        clinical_df=clinical_raw,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    print("Clinical features:")
    print(clinical_features)
    
    # Save clinical features to file
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    clinical_features.to_csv(out_clinical_data_file)
    print(f"Clinical features saved to {out_clinical_data_file}")
    
    # 3. Link clinical and genetic data
    linked_data = geo_link_clinical_genetic_data(clinical_features, normalized_gene_data)
    print(f"Linked data shape: {linked_data.shape}")
    print("Linked data preview (first 5 rows, first 5 columns):")
    print(linked_data.iloc[:5, :5])
    
    # 4. Handle missing values
    print("Missing values before handling:")
    print(f"  Trait ({trait}) missing: {linked_data[trait].isna().sum()} out of {len(linked_data)}")
    if 'Age' in linked_data.columns:
        print(f"  Age missing: {linked_data['Age'].isna().sum()} out of {len(linked_data)}")
    if 'Gender' in linked_data.columns:
        print(f"  Gender missing: {linked_data['Gender'].isna().sum()} out of {len(linked_data)}")
    print(f"  Genes with >20% missing: {sum(linked_data.iloc[:, 1:].isna().mean() > 0.2)}")
    print(f"  Samples with >5% missing genes: {sum(linked_data.iloc[:, 1:].isna().mean(axis=1) > 0.05)}")
    
    cleaned_data = handle_missing_values(linked_data, trait)
    print(f"Data shape after handling missing values: {cleaned_data.shape}")
    
    # 5. Evaluate bias in trait and demographic features
    is_trait_biased = False
    if len(cleaned_data) > 0:
        trait_biased, cleaned_data = judge_and_remove_biased_features(cleaned_data, trait)
        is_trait_biased = trait_biased
    else:
        print("No data remains after handling missing values.")
        is_trait_biased = True
    
    # 6. Final validation and save
    is_usable = validate_and_save_cohort_info(
        is_final=True, 
        cohort=cohort, 
        info_path=json_path, 
        is_gene_available=True, 
        is_trait_available=True, 
        is_biased=is_trait_biased, 
        df=cleaned_data,
        note="Dataset contains gene expression data comparing CFTR WT vs CFTR mutant (p.Phe508del) samples."
    )
    
    # 7. Save if usable
    if is_usable and len(cleaned_data) > 0:
        os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
        cleaned_data.to_csv(out_data_file)
        print(f"Linked data saved to {out_data_file}")
    else:
        print("Data was determined to be unusable or empty and was not saved")
        
except Exception as e:
    print(f"Error processing data: {e}")
    # Handle the error case by still recording cohort info
    validate_and_save_cohort_info(
        is_final=True, 
        cohort=cohort, 
        info_path=json_path, 
        is_gene_available=True, 
        is_trait_available=False,  # Mark as not available due to processing issues
        is_biased=True, 
        df=pd.DataFrame(),  # Empty dataframe
        note=f"Error processing data: {str(e)}"
    )
    print("Data was determined to be unusable and was not saved")